
# **Berechnung der Beobachterverstärkung für einen Kondensatorstrombeobachter**

Dieses Notebook berechnet die Verstärkungen \( L_1 \) und \( L_2 \) für einen Luenberger-Beobachter,
der den Kondensatorstrom \( I_C \) aus der Spannung \( V \) rekonstruiert.

## **1. Systembeschreibung**
Das System besteht aus einem Kondensator mit Kapazität \( C \) und einem Innenwiderstand \( R_s \).
Die einzige messbare Größe ist die Spannung \( V \), aus der der Strom \( I_C \) rekonstruiert werden soll.

Die Grundgleichungen sind:

\begin{equation}
I_C = C \frac{dV_C}{dt}
\end{equation}

\begin{equation}
V = V_C + R_s I_C.
\end{equation}

Definiert man die Zustandsvariablen als:

\begin{equation}
x_1 = V_C, \quad x_2 = I_C,
\end{equation}

ergibt sich die Zustandsraumdarstellung:

\begin{equation}
\dot{x} =
\begin{bmatrix} 
0 & \frac{1}{C} \\ 
0 & 0 
\end{bmatrix} x,
\end{equation}

\begin{equation}
y = 
\begin{bmatrix} 1 & R_s \end{bmatrix} x.
\end{equation}

Da nur die Spannung \( y = V \) messbar ist, wird ein Beobachter zur Schätzung von \( I_C \) entworfen.



## **2. Systemparameter**
Für die Berechnung der Verstärkungen werden folgende Werte angenommen:

- Kapazität: \( C = 50 \) F
- Innenwiderstand: \( R_s = 22.0 \) mΩ
- Kondensatorspannung: \( V = 3 \) V

Diese Werte beeinflussen die numerische Berechnung, jedoch nicht die symbolische Herleitung.



## **3. Fehlerdynamik und Eigenwerte**
Die Fehlerdynamik wird beschrieben durch:

\begin{equation}
\dot{e} = (A - L C) e.
\end{equation}

Mit:

\begin{equation}
A - L C =
\begin{bmatrix} 
- L_1 & \frac{1}{C} - L_1 R_s \\ 
- L_2 & - L_2 R_s
\end{bmatrix}.
\end{equation}

Die charakteristische Gleichung lautet:

\begin{equation}
\lambda^2 + (L_1 + L_2 R_s) \lambda + L_1 L_2 R_s - L_2 \frac{1}{C} = 0.
\end{equation}

Für eine stabile Beobachterdynamik setzen wir doppelte Pole bei \( \lambda = -\alpha \):

\begin{equation}
(\lambda + \alpha)^2 = \lambda^2 + 2\alpha \lambda + \alpha^2.
\end{equation}

Daraus ergeben sich die Gleichungen zur Bestimmung von \( L_1 \) und \( L_2 \).



### **3.1 Symbolische Berechnung der Verstärkungen**

Die Verstärkungen \( L_1 \) und \( L_2 \) ergeben sich aus:

\begin{equation}
L_1 + L_2 R_s = 2 \alpha
\end{equation}

\begin{equation}
L_1 L_2 R_s - L_2 \frac{1}{C} = \alpha^2.
\end{equation}

Symbolische Berechnung mit **SymPy**:


In [9]:

import sympy as sp

# Definiere Systemparameter als Symbole
C, Rs, alpha, L1, L2 = sp.symbols('C Rs alpha L1 L2', positive=True, real=True)

# Charakteristische Gleichung für den Beobachter
char_eq = sp.Eq(L1 + L2 * Rs, 2 * alpha)
char_eq2 = sp.Eq(L1 * L2 * Rs - L2 / C, alpha**2)

# Löse nach L1 und L2
solutions = sp.solve([char_eq, char_eq2], (L1, L2), dict=True)

# Formatierte Ausgabe der Lösungen
for sol in solutions:
    display(sol)


{L1: -Rs*(-sqrt(-4*C*Rs*alpha + 1)/(2*C*Rs**2) + (2*C*Rs*alpha - 1)/(2*C*Rs**2)) + 2*alpha,
 L2: -sqrt(-4*C*Rs*alpha + 1)/(2*C*Rs**2) + (2*C*Rs*alpha - 1)/(2*C*Rs**2)}

{L1: -Rs*(sqrt(-4*C*Rs*alpha + 1)/(2*C*Rs**2) + (2*C*Rs*alpha - 1)/(2*C*Rs**2)) + 2*alpha,
 L2: sqrt(-4*C*Rs*alpha + 1)/(2*C*Rs**2) + (2*C*Rs*alpha - 1)/(2*C*Rs**2)}


### **3.2 Wahl des Konvergenzfaktors \( \alpha \)**
Um die Konvergenz des Beobachters sinnvoll zu steuern, setzen wir:

\begin{equation}
\alpha = \frac{10}{R_s C}.
\end{equation}

Dies bewirkt, dass der Beobachter etwa **10-mal schneller als die physikalische Zeitkonstante** \( R_s C \) konvergiert.



### **3.3 Numerische Berechnung der Verstärkungen**

Setzt man die gegebenen Systemparameter ein:


In [10]:

# Numerische Werte definieren
C_value = 50  # Kapazität in Farad
Rs_value = 0.022  # Innenwiderstand in Ohm

# Alpha numerisch berechnen
alpha_value = 10 / (Rs_value * C_value)

# Berechne L1 und L2 mit eingesetztem Alpha
solutions_numeric = [{L1: sol[L1].subs({C: C_value, Rs: Rs_value, alpha: alpha_value}), 
                      L2: sol[L2].subs({C: C_value, Rs: Rs_value, alpha: alpha_value})} 
                     for sol in solutions]

# Formatierte Ausgabe der numerischen Lösungen
for sol in solutions_numeric:
    display(sol)


{L1: 9.54545454545455 + 2.83863545381745*I,
 L2: 392.561983471074 - 129.02888426443*I}

{L1: 9.54545454545455 - 2.83863545381745*I,
 L2: 392.561983471074 + 129.02888426443*I}


## **4. Fazit**
Dieses Notebook leitet die symbolischen Formeln für die Verstärkungen \( L_1 \) und \( L_2 \) her und berechnet diese anschließend numerisch für ein gegebenes System.

Die Berechnung zeigt, dass die Verstärkungen von der Kapazität \( C \), dem Innenwiderstand \( R_s \) und der gewählten Konvergenzgeschwindigkeit \( \alpha \) abhängen.
Eine schnelle Konvergenz kann durch eine größere Wahl von \( \alpha \) erzielt werden.
